In [51]:
import numpy as np
import pandas as pd
import requests
import re

In [3]:
!head -n5 P3_GrantExport.csv

In [6]:
pd.read_csv?

In [287]:
df = pd.read_csv('P3_GrantExport.csv', sep=';', index_col=0, na_values=['Nicht zuteilbar - NA'])
df.head()

,Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
Project Number,,,,,,,,,,,,,,
1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,NaN,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619.00,NaN
4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.00,NaN
5,"Kritische Erstausgabe der ""Evidentiae contra D...",NaN,Kommission für das Corpus philosophorum medii ...,Project funding (Div. I-III),Project funding,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732.00,NaN
6,Katalog der datierten Handschriften in der Sch...,NaN,Burckhardt Max,Project funding (Div. I-III),Project funding,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,52627.00,NaN
7,Wissenschaftliche Mitarbeit am Thesaurus Lingu...,NaN,Schweiz. Thesauruskommission,Project funding (Div. I-III),Project funding,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",10303,Ancient history and Classical studies,Human and Social Sciences;Theology & religious...,01.01.1976,30.04.1978,120042.00,NaN


In [13]:
df.index.is_unique

True

In [98]:
match_format = df.University.dropna().str.match('^.+ - .+$', as_indexer=True)

In [14]:
df.University.describe()

count                       50988
unique                         77
top       Universität Zürich - ZH
freq                         6774
Name: University, dtype: object

In [167]:
def trim_university_name(full_name):
    match = re.match('^([\s\w]+) - \w+$', full_name)
    if match is None:
        return None
    else:
        return match.group(1)

In [321]:
def get_info(university_name):
    parameters = {'username': 'ada_epfl', 'country': 'CH', 'name': trim_university_name(university_name),
                 'orderby': 'relevance', 'isNameRequired': 'true'}
    r = requests.get('http://api.geonames.org/searchJSON', params=parameters)
    
    results = r.json().get('geonames')

    if results is None or not results: # if None or empty
        return np.nan
    else:
        first = results[0]
        return (first['adminCode1'], first['lat'], first['lng'])

In [322]:
get_info(df.iloc[3,6])

('BS', '47.55832', '7.58403')

In [323]:
get_info(df.loc[6076,'University'])

nan

In [324]:
get_info(df.iloc[5,6])

nan

In [325]:
uni_infos = {university: get_info(university) for university in df.University.dropna().unique()}

In [328]:
unis.count()

canton       5
latitude     5
longitude    5
dtype: int64

In [327]:
unis = pd.DataFrame(uni_infos).T
unis.columns = ['canton', 'latitude', 'longitude']
unis.index.name = 'university'
unis

,canton,latitude,longitude
university,,,
AO Research Institute - AORI,NaN,NaN,NaN
Allergie- und Asthmaforschung - SIAF,NaN,NaN,NaN
Berner Fachhochschule - BFH,NaN,NaN,NaN
Biotechnologie Institut Thurgau - BITG,NaN,NaN,NaN
Centre de rech. sur l'environnement alpin - CREALP,NaN,NaN,NaN
EPF Lausanne - EPFL,NaN,NaN,NaN
ETH Zürich - ETHZ,ZH,47.3763,8.54805
Eidg. Anstalt für Wasserversorgung - EAWAG,NaN,NaN,NaN
"Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL",NaN,NaN,NaN


In [275]:
unis.to_csv('universities.csv')

In [276]:
!head -n5 universities.csv

university,canton,latitude,longitude
AO Research Institute - AORI,,,
Allergie- und Asthmaforschung - SIAF,BE,46.94809,7.44744
Berner Fachhochschule - BFH,,,
Biotechnologie Institut Thurgau - BITG,,,


In [199]:
uni_cantons[uni_cantons.isnull()]

university
AO Research Institute - AORI                          NaN
Berner Fachhochschule - BFH                           NaN
Biotechnologie Institut Thurgau - BITG                NaN
EPF Lausanne - EPFL                                   NaN
Ente Ospedaliero Cantonale - EOC                      NaN
Fachhochschule Kalaidos - FHKD                        NaN
Fachhochschule Ostschweiz - FHO                       NaN
Facoltà di Teologia di Lugano - FTL                   NaN
Forschungsanstalten Agroscope - AGS                   NaN
Forschungsinstitut für Opthalmologie - IRO            NaN
Forschungsinstitut für biologischen Landbau - FIBL    NaN
Franklin University Switzerland - FUS                 NaN
Friedrich Miescher Institute - FMI                    NaN
Haute école pédagogique du canton de Vaud - HEPL      NaN
Haute école pédagogique fribourgeoise - HEPFR         NaN
Hochschule Luzern - HSLU                              NaN
Idiap Research Institute - IDIAP                      NaN
Ins

In [191]:
len(uni_cantons)

77

In [196]:
len(df.University.unique())

78